# Figanos multiple plots

Create subplots and facetgrids with Figanos.

In [ ]:
# import necessary libraries
import xarray as xr
import cartopy.crs as ccrs
import figanos.matplotlib as fg
import numpy as np

# use ouranos style
fg.utils.set_mpl_style('ouranos')

In [ ]:
# create xarray object from a NetCDF
url = 'https://pavics.ouranos.ca//twitcher/ows/proxy/thredds/dodsC/birdhouse/disk2/cccs_portal/indices/Final/BCCAQv2_CMIP6/tx_max/YS/ssp585/ensemble_percentiles/tx_max_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc'
opened = xr.open_dataset(url, decode_timedelta=False)

## Timeseries
Create multiple timeseries plot with matplotlib subplots and figanos as shown below.

## Maps
Create multiple maps plot with figanos wrapped around xr.plot.facetgrid.FacetGrid by passing the arguement row and/or col in plot_kw.

In [ ]:
#Selecting a time and slicing our starting Dataset
ds_space = opened[['tx_max_p50']].isel(time=[0, 1, 2]).sel(lat=slice(40,65), lon=slice(-90,-55))

# defining our projection.
projection = ccrs.LambertConformal()

im = fg.gridmap(ds_space, projection = projection, plot_kw = {"x": "lon", "y": "lat", "col": "time"}, features = ['coastline','ocean'], frame = False, use_attrs={"suptitle": "description"})



In [ ]:
names = ['station_' + str(i) for i in np.arange(5)]
lat = 45 + np.random.rand(5)*3
lon = np.linspace(-76,-70, 5)
tas = np.array([[20, 25, 30, 15, 5], [5, 0, 10, 2, 3]])
yrs = np.array([[35, 65, 45, 25, 95],
                [15, 75, 10, 15, 50]])

attrs = {'units': 'degC', 'standard_name': 'air_temperature', 'long_name': 'Near-Surface Daily Maximum Air Temperature'}

tas = xr.DataArray(data=tas,
                        coords={'season': ['DFJ', 'MAM'],
                                'station': names,
                                'lat':('station', lat),
                                'lon': ('station', lon),
                                'years': (('season', 'station'), yrs),
                                },
                        dims=['season', 'station'],
                        attrs=attrs)
obs = xr.Dataset({'tas': tas})

# plot
fg.scattermap(obs,
                  transform=ccrs.PlateCarree(),
                  sizes='years',
                  size_range=(25, 100),
                  plot_kw={
                      "xlim": (-77,-69),
                      "ylim":(43,50),
                      "col": "season",
                  },
                  features={
                      "land": {"color": "#f0f0f0"},
                      "rivers": {"edgecolor": "#cfd3d4"},
                      "lakes": {"facecolor": "#cfd3d4"},
                      "coastline": {"edgecolor": "black"},
                  }
                 )

## Plot over each other

In [ ]:
names = ['station_' + str(i) for i in np.arange(5)]
lat = 45 + np.random.rand(5)*3
lon = np.linspace(-76,-70, 5)
tas = np.array([[290, 300, 295, 305, 301], [275, 285, 277, 301, 345], [302, 293, 295, 292, 280]])

attrs = {'units': 'degK', 'standard_name': 'air_temperature', 'long_name': ds_space.tx_max_p50.attrs['description']}

tas = xr.DataArray(data=tas,
                        coords={'time': ds_space.time.values,
                                'station': names,
                                'lat':('station', lat),
                                'lon': ('station', lon),
                                },
                        dims=['time', 'station'],
                        attrs=attrs)
obs2 = xr.Dataset({'tas': tas})

In [ ]:
vmin=280
vmax=310

im = fg.gridmap(ds_space,
                projection = projection,
                plot_kw = {"col": "time",
                           "xlim": (-77,-69), "ylim": (43,50),
                           "vmin": vmin, "vmax": vmax,
                           },
                features = ['coastline','ocean'],
                frame = False,
                use_attrs={"suptitle": "description"}
                )
for i, fax in enumerate(im.axs.flat):
    fg.scattermap(obs2.isel(time=i),
                  ax=fax,
                  transform=ccrs.PlateCarree(),
                  plot_kw={'vmin': vmin, 'vmax': vmax, 'edgecolors':'grey', 'add_colorbar': False}
                 )
